# Data import

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
FILES_DIR = './files/'
events = pd.read_csv(FILES_DIR + 'sampled_events.csv')
product = pd.read_csv(FILES_DIR + 'sampled_products.csv')
users = pd.read_csv(FILES_DIR + 'sampled_users.csv')

In [21]:
print(events.shape)
print(product.shape)
print(users.shape)

(5880407, 13)
(283326, 12)
(228041, 4)


In [5]:
# 리소스 문제로 29402 데이터만 사용
# duplicated 하는 이유는 log데이터 특성상
events_df = events.copy()
events_sample = events_df.sample(frac=0.005, random_state=42)

,session_id,event_timestamp,event_name,user_no,item_no,device_type,mobile_brand_name,mobile_model_name,mobile_marketing_name,operating_system_version,country,region,platform
1916969,434cf67167c5becc6321d03bdecd6c60,2021-06-24 09:25:17.097027,click_item,qwTrxFylir2ynPu9N7/g/w==,/ZasVJSuvWNKGQXzKT0uCg==,mobile,Apple,iPhone,NaN,iOS 14.4.2,South Korea,Seoul,IOS
4178142,23288f3e8d799799645beb4bff6c35ad,2021-07-16 22:38:19.169111,click_item,OXbjPQaxKL7O8HoSMzJtiQ==,/YlZZk+EM3eAaLCGdgyiag==,mobile,Apple,iPhone,NaN,iOS 14.6,South Korea,Seoul,IOS
644115,dc7c3da68a67b1f43a2226ea2733bebf,2021-06-09 23:25:54.245213,click_item,1jhscZ7MrX7w6sojiWm8cw==,TByIyj8aQeeoE2OnGWr4NQ==,mobile,Samsung,SM-N981N,Galaxy Note20 5G,Android 11,South Korea,Seoul,ANDROID
2047331,19422b436106a1cbc45bbe12bc59d11a,2021-06-25 22:37:42.032078,click_item,6+Uf31nW96b3VC61EfO4SQ==,F+25W5Hy9tVYnBlffA2Cmw==,mobile,Apple,iPhone,NaN,iOS 14.6,South Korea,Chungcheongnam-do,IOS
2679735,1a963aba0fb154be005ab97ddbf780aa,2021-07-02 08:51:53.27311,click_item,Zfy9Vola1OIOtuunlXev+A==,VoEeDect6p+wMXMoE5zAkw==,mobile,Samsung,SM-G965N,Galaxy S9+,Android 10,South Korea,Seoul,ANDROID
...,...,...,...,...,...,...,...,...,...,...,...,...,...
594921,4b2b5f6db5589fa7f736b0bd31f87685,2021-06-09 14:33:30.903094,click_item,3df2n7oVw8oZ3MuLxK5KYg==,GD6DtsasqVJamffP55apQA==,mobile,Apple,iPhone,NaN,iOS 14.6,South Korea,Gyeonggi-do,IOS
1256645,288d50850a8b0e43f2f2e6226a33e53a,2021-06-17 08:53:13.703111,click_item,2d9zQSt+DExdfvP/mj3Juw==,OCVwIGJhWNYNIwAxdI/Lag==,mobile,Samsung,SM-G977N,Galaxy S10 5G,Android 11,South Korea,Seoul,ANDROID
4173205,95cdce3d28de536e37dc837cedb58f21,2021-07-16 21:55:49.05804,add_to_cart,fxAptePuBEdVPekHLnmFnw==,mAznRa3p9hv4rhWkunWMFQ==,mobile,Samsung,SM-N960N,Galaxy Note9,Android 10,South Korea,Gyeonggi-do,ANDROID
3952661,5efe98cef53495a0ccec290e57e367fa,2021-07-14 17:17:28.635075,click_item,0h0lKPYrJcPeQlwIjKRohg==,tKZFh9pXCjI7VKLT3Pt1tw==,mobile,Apple,iPhone,NaN,iOS 14.6,South Korea,Jeollabuk-do,IOS


In [ ]:
# 의미없는 컬럼 삭제 및 null값 처리
drop_events = events_sample.copy()
drop_events['mobile_brand_name'].fillna('un_brand_name', inplace=True)
drop_events.drop('mobile_marketing_name', axis=1, inplace=True)
drop_events.drop('mobile_model_name', axis=1, inplace=True)
drop_events.drop('operating_system_version', axis=1, inplace=True)
drop_events['rating'] = drop_events['event_name']
re_name = {'rating' : {'click_item':1, 'like_item':2, 'add_to_cart':3, 'purchase_success':4}}
drop_events = drop_events.replace(re_name)
drop_events.dropna(axis=0, inplace=True)
df_events = drop_events.copy()
df_events

In [ ]:
print(df_events.isnull().sum().sort_values(ascending=False))

In [11]:
today = pd.Timestamp.now().floor('D')
users['birth_date'] = pd.to_datetime(users['birth_date'], errors='coerce')
users = users[~pd.isnull(users['birth_date'])] # remove rows with missing birth date
users['age'] = ((today - users['birth_date']).dt.days / 365.25).astype(int)

def age_category(age):
    age = (age // 10) * 10
    return age

users['age'] = users.age.apply(age_category)

In [13]:
total_df = pd.merge(df_events, users)
total_df = pd.merge(total_df, product)
total_df['birth_date'].fillna('un_birth_date', inplace=True)
total_df['gender'].fillna('un_gender', inplace=True)
total_df.isnull().sum()

session_id           0
event_timestamp      0
event_name           0
user_no              0
item_no              0
device_type          0
mobile_brand_name    0
country              0
region               0
platform             0
rating               0
birth_date           0
gender               0
age                  0
item_name            0
image_name           0
price                0
category1_code       0
category1_name       0
category2_code       0
category2_name       0
category3_code       0
category3_name       0
brand_no             0
brand_name           0
dtype: int64

In [14]:
def recom_item(item):
    return total_df.loc[product_mean.sort_values(by='rating', ascending=False)[:item].index]['item_name']

product_mean = total_df.groupby(['item_no'])['rating'].mean()
product_mean = product_mean.reset_index()
recom_item(5)

11326                           크링클 우디 팬츠 (Musky Black)
15392              플라워 나눔 플레이트 3color / 만두접시 찬기 초밥 소스접시
9638     MARITHE X DIAGONAL WAVY BUTTON T-SHIRT (white)
4334          SALT-WATER SANDAL ADULT CLASSIC SLIDE TAN
9639     MARITHE X DIAGONAL WAVY BUTTON T-SHIRT (white)
Name: item_name, dtype: object

In [17]:
# train, test split
x = total_df.copy()
y = total_df['user_no']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

x_train.drop_duplicates(subset=['user_no', 'item_no'], inplace=True)


def RMSE(y_true, y_pred):
  return np.sqrt(np.mean(np.power(np.array(y_true)-np.array(y_pred), 2)))

def score(model):
  id_pairs = zip(x_test['user_no'], x_test['item_no'])
  y_pred = np.array([model(user, item) for (user, item) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)



rating_matrix = x_train.pivot(index='user_no', columns='item_no', values='rating')

In [22]:
def best_seller(user_no, item_no):
  try:
    rating = train_mean[item_no]
  except KeyError:
    rating = 2.5
  return rating

train_mean = x_train.groupby(['item_no'])['rating'].mean()
print('best-seller 방식의 정확도:', score(best_seller))

best-seller 방식의 정확도: 1.3118513888842565
